In [1]:
import tkinter as tk  # 使用Tkinter前需要先导入
from tkinter.filedialog import askdirectory
import camera
import classifier
import hand_eye
import geo_demo

window = tk.Tk()
window.title('基于RGB-D相机的机械臂分类抓取系统')
window.geometry('1080x720')  # 这里的乘是小x

def kong():
    pass

def run_1():
    window.withdraw()
    camera.do_camera(window)
def run_2():
    window.withdraw()
    classifier.do_classfier(window)
def run_3():
    window.withdraw()
    hand_eye.do_hand_eye(window)
def run_4():
    window.withdraw()
    geo_demo.do_geo_demo(window)

canvas = tk.Canvas(window, bg='green', height=720, width=1080)

image_file = tk.PhotoImage(file='imgs/2.png')  # 图片位置（相对路径，与.py文件同一文件夹下，也可以用绝对路径，需要给定图片具体绝对路径）
image = canvas.create_image(540, 0, anchor='n',image=image_file)

canvas.pack()

tk.Label(window, text='基于RGB-D相机的机械臂分类抓取系统',bg='red', font=('黑体', 30), ).place(x=540, y=100, anchor='n')
a = tk.Button(window, text="数据采集", font=('黑体', 20),bg='yellow', command=run_1,width=20,height=2)
a.place(x=300, y=300, anchor='n')
b = tk.Button(window, text="分类器构建", font=('黑体', 20),bg='yellow', command=run_2,width=20,height=2).place(x=800, y=300, anchor='n')
c = tk.Button(window, text="手眼标定", font=('黑体', 20),bg='yellow', command=run_3,width=20,height=2).place(x=300, y=500, anchor='n')
d = tk.Button(window, text="分类抓取", font=('黑体', 20),bg='yellow', command=run_4,width=20,height=2).place(x=800, y=500, anchor='n')

# t = tk.Text(window, height=15,width=100)
# t.place(x=540, y=400, anchor='n')
#
#
#
#
# t.tag_add("tag1", "1.7", "1.13", "1.15")
# t.tag_config("tag1", background="yellow", foreground="red", font=('黑体', 20))
# t.tag_config("1.7",   font=('黑体', 20))
# t.insert("insert", "说明：",'tag1')
# t.insert("insert", "\n")
# t.insert("insert", "本系统可实现对物体的分类抓取操作，共分为三大模块：\n",'1.7')
# t.insert("insert", "分类器构建，手眼标定和分类抓取\n",'1.7')
# t.insert("insert", "请按照顺序依次先完成分类器构建与手眼标定\n",'1.7')
# t.insert("insert", "再开始分类抓取操作\n",'1.7')
# t.insert("insert", "感谢对本系统的使用！\n",'1.7')

window.mainloop()

ModuleNotFoundError: No module named 'camera'

In [2]:
import tkinter as tk  # 使用Tkinter前需要先导入
import tkinter.filedialog as tkf
import pyrealsense2 as rs
import numpy as np
import cv2
import threading
import os
from PIL import Image,ImageTk
import pickle

def do_camera(parent):
    window = tk.Toplevel()
    window.title('数据采集')
    window.geometry('1080x1000')  # 这里的乘是小x

    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
    pipeline.start(config)

    def kong():
        pass
    global canvas1
    canvas1 = tk.Label(window, width=640, height=960)
    canvas1.place(x=250,y=50,anchor='nw')

    def selectPath():
        global path_
        cnt=1
        path_=tkf.asksaveasfilename()
        path_=os.path.normpath(path_)
        path_=os.path.split(path_)[0]
        print(path_)
        print(os.path.join(path_, str(cnt) + '.jpg'))

    def take_photo():
        global path_
        global cnt
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        color_image = np.asanyarray(color_frame.get_data())
        print("lujing:",path_)
        cv2.imwrite(os.path.join(path_, str(cnt) + '.jpg'),color_image)
        cnt=cnt+1

    global cnt
    cnt=1
    with open('bayes1.pkl', 'rb') as f:
        bc = pickle.load(f)
    path_='H:/Python/GUI/data'
    var1 = tk.IntVar()
    c1 = tk.Checkbutton(window, text='图像对齐',font=('黑体', 20),variable=var1, onvalue=1, offvalue=0, command=kong)    # 传值原理类似于radiobutton部件
    c1.place(x=20, y=400, anchor='nw')
    tk.Label(window, text='彩色图像',bg='LightSkyBlue', font=('黑体', 20), ).place(x=900, y=250, anchor='nw')
    tk.Label(window, text='深度图像',bg='LightSkyBlue', font=('黑体', 20), ).place(x=900, y=750, anchor='nw')
    b = tk.Button(window, text="路径选择", font=('黑体', 20),bg='yellow', command=selectPath,width=12,height=1)
    b.place(x=20, y=200, anchor='nw')
    c = tk.Button(window, text="开始拍摄", font=('黑体', 20),bg='yellow', command=take_photo,width=12,height=1)
    c.place(x=20, y=300, anchor='nw')

    def start():
        a.config(bg='green')
        while True:
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            color_frame = frames.get_color_frame()
            if not depth_frame or not color_frame:
                continue

            # Convert images to numpy arrays

            depth_image = np.asanyarray(depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())

            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.3), cv2.COLORMAP_JET)
            # Stack both images horizontally

            if (var1.get() == 1):
                ali_dep = rs.align(rs.stream.depth)
                ali_col = rs.align(rs.stream.color)
                frameset = ali_col.process(frames)
                depth_frame = frameset.get_depth_frame()
                color_frame = frameset.get_color_frame()
                depth_image = np.asanyarray(depth_frame.get_data())
                # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
                depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.3), cv2.COLORMAP_JET)
                # Stack both images horizontally

            images = np.vstack((color_image, depth_colormap))
            images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
            image2 = Image.fromarray(images)
            img = ImageTk.PhotoImage(image2)
            canvas1.configure(image=img)
            # Show images
            window.update_idletasks()
            window.update()
            key = cv2.waitKey(1)


    tk.Label(window, text='数据采集',bg='red', font=('黑体', 30), ).pack(side='top')
    tk.Label(window, text='彩色图像',bg='LightSkyBlue', font=('黑体', 20), ).place(x=900, y=250, anchor='nw')
    tk.Label(window, text='深度图像',bg='LightSkyBlue', font=('黑体', 20), ).place(x=900, y=750, anchor='nw')
    a = tk.Button(window, text="开启摄像头", font=('黑体', 20),bg='yellow', command=start,width=12,height=1)
    a.place(x=20, y=100, anchor='nw')

    t = tk.Text(window, height=35,width=30)
    t.place(x=20, y=450, anchor='nw')
    t.tag_add("tag1", "1.7", "1.13", "1.15")
    t.tag_config("tag1", background="yellow", foreground="red", font=('黑体', 20))
    t.tag_config("1.7",   font=('黑体', 18))
    t.insert("insert", "提示：",'tag1')
    t.insert("insert", "\n")
    t.insert("insert", "按下“开启摄像头”按钮，摄像头开始传输图像，主窗口显示彩色和深度图像\n",'1.7')
    t.insert("insert", "按下“路径选择”按钮，便可以选择拍摄图片保存的路径\n",'1.7')
    t.insert("insert", "按下“开始拍摄”按钮，便可以拍摄当前图像并保存到所选择的路径中\n",'1.7')
    t.insert("insert", "勾选“图像对齐”按钮，便可以检查图像对齐功能是否正常\n",'1.7')
    #t.insert("insert", "目前已标定点个数：0\n",'1.7')
    #t.insert("insert", "剩余标定点个数：6\n",'1.7')




    # t = tk.Text(window, height=30,width=30)
    # t.place(x=50, y=200, anchor='nw')
    #
    #
    # t.tag_add("tag1", "1.7", "1.13", "1.15")
    # t.tag_config("tag1", background="yellow", foreground="red", font=('黑体', 20))
    # t.tag_config("1.7",   font=('黑体', 20))
    # t.insert("insert", "说明：",'tag1')
    # t.insert("insert", "\n")
    # t.insert("insert", "该窗口用于演示图像对齐效果，勾选图像对齐选项框便可看到图像对齐后彩色图像和深度图像\n",'1.7')

    def back():
        pipeline.stop()
        window.destroy()
        parent.deiconify()

    tui=tk.Button(window, text="返回", font=('黑体', 20),bg='yellow', command=back,width=5,height=1)
    tui.place(x=800, y=0, anchor='nw')

    window.mainloop()

if __name__ == '__main__':
    do_camera(1)

ModuleNotFoundError: No module named 'pyrealsense2'